# GenoMix Mamba 2

In [2]:
import sys

from transformers import Mamba2Config
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

sys.path.append('..')

from genomix.models.genomix_modeling.genomix_mamba2 import GenomixMamba2


In [15]:
config = Mamba2Config()
config.d_model = 512
config.n_head = 8
config.n_layer = 24
config.vocab_size=16
config.hidden_size=512
config.intermediate_size=512
config.num_heads=8
config.num_layers=24
config.tie_word_embeddings=True

print(config)

Mamba2Config {
  "bos_token_id": 0,
  "chunk_size": 256,
  "conv_kernel": 4,
  "d_model": 512,
  "eos_token_id": 2,
  "expand": 2,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 512,
  "initializer_range": 0.1,
  "intermediate_size": 512,
  "layer_norm_epsilon": 1e-05,
  "model_type": "mamba2",
  "n_groups": 8,
  "n_head": 8,
  "n_layer": 24,
  "num_heads": 8,
  "num_hidden_layers": 64,
  "num_layers": 24,
  "pad_token_id": 1,
  "rescale_prenorm_residual": false,
  "residual_in_fp32": true,
  "rms_norm": true,
  "state_size": 128,
  "tie_word_embeddings": true,
  "time_step_floor": 0.0001,
  "time_step_limit": [
    0.0,
    Infinity
  ],
  "time_step_max": 0.1,
  "time_step_min": 0.001,
  "time_step_rank": 256,
  "transformers_version": "4.46.3",
  "use_bias": false,
  "use_cache": true,
  "use_conv_bias": true,
  "vocab_size": 16
}



In [16]:
model = GenomixMamba2(config)

In [17]:
from genomix.utils import model_size

In [18]:
model_size(model)

169125888

In [14]:
model.base_model_prefix

'model.backbone'